In [1]:
import json

In [3]:
with open('../linux-commits-2023-11-12.json') as f:
    all_commits = f.readlines()

In [ ]:
all_commits_dict_list = []
for commit in all_commits:
    commit_dict = json.loads(commit)
    all_commits_dict_list.append(commit_dict)

In [ ]:
len(all_commits_dict_list)